In [ ]:
import os
import uuid
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

# ✅ 設定 OpenAI 金鑰
os.environ["OPENAI_API_KEY"] = "api key"

# ✅ 讀取 PDF
pdf_dir = "./PDF"  # ← 確保這個資料夾存在且有 PDF
all_documents = []

for file in os.listdir(pdf_dir):
    if file.lower().endswith(".pdf"):
        file_path = os.path.join(pdf_dir, file)
        loader = PyPDFLoader(file_path)
        pages = loader.load()
        all_documents.extend(pages)

# ✅ 分割文本
splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
documents = splitter.split_documents(pages)

# ✅ 建立向量資料庫 (Chroma)
persist_directory = "./chroma_pdf_db"
embedding_model = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(
    documents,
    embedding_model,
    persist_directory=persist_directory,
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# ✅ 建立提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一位材料科學助理，請根據以下內容回答問題。\n\n{context}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# ✅ 建立 LLM 與文件鏈
llm = ChatOpenAI(model_name="gpt-4o", temperature=0, max_tokens=8192)
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# ✅ 定義 SQL-based 記憶儲存
def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///./langchain.db")

chain_with_history = RunnableWithMessageHistory(
    retrieval_chain,
    get_session_history,
    input_messages_key="input",
    output_messages_key="answer",
    history_messages_key="history"
)

# ✅ 啟動對話（測試）
session_id = str(uuid.uuid4())  # 每次執行都用新的 session，可改為固定值測試
config = {"configurable": {"session_id": session_id}}

response1 = chain_with_history.invoke({"input": "請問在目前的高熵合金應用中，哪些元素組合常見於催化劑？"}, config=config)
print("🤖 回答1：", response1["answer"])

response2 = chain_with_history.invoke({"input": "input": "哪一組的表現比較好？"}, config=config)
print("🤖 回答2：", response2["answer"])


C:\Users\chen\AppData\Local\Temp\ipykernel_68548\3587039507.py:35: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings()
C:\Users\chen\AppData\Local\Temp\ipykernel_68548\3587039507.py:53: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o", temperature=0, max_tokens=8192)
c:\Users\chen\anaconda3\envs\langchainragas\lib\si

🤖 回答1： 這份文件主要討論了高熵合金（HEA）塗層的微觀結構和耐磨性能，特別是通過TIG覆蓋技術製備的CoCrFeMnNbNi高熵合金塗層。文件中比較了AISI 304鋼和高熵合金塗層在磨損表面上的微觀結構，指出高熵合金塗層在乾滑動磨損條件下具有優異的耐磨性，並且其磨損表面的微犁現象較淺且較細，這與Laves相的納米尺度層狀結構的形成有關。文件還提到這種塗層的組成變化對其性能的影響。
🤖 回答2： 文件中提到的金屬材料包括鈷（Co）、鉻（Cr）、鐵（Fe）、錳（Mn）、鈮（Nb）和鎳（Ni）。這些金屬被用來製備高熵合金塗層。


In [4]:
# ✅ 啟動對話（測試）
session_id = str(uuid.uuid4())  # 每次執行都用新的 session，可改為固定值測試
config = {"configurable": {"session_id": session_id}}

response1 = chain_with_history.invoke({"input": "請問在目前的高熵合金應用中，哪些元素組合常見於催化劑？"}, config=config)
print("🤖 回答1：", response1["answer"])

response2 = chain_with_history.invoke({"input":"哪一組的表現比較好？"}, config=config)
print("🤖 回答2：", response2["answer"])

🤖 回答1： 在高熵合金（HEAs）應用於催化劑的研究中，常見的元素組合包括過渡金屬元素，如鎳（Ni）、鈷（Co）、鐵（Fe）、銅（Cu）、鉻（Cr）、錳（Mn）等。這些元素的組合可以形成具有多樣性和穩定性的合金系統，這些特性對於催化劑的性能提升非常有利。此外，某些研究也會加入貴金屬元素如鉑（Pt）、鈀（Pd）等，以進一步提高催化活性和選擇性。這些高熵合金催化劑在多相催化、電催化和光催化等領域展現出潛力。
🤖 回答2： 高熵合金在催化劑應用中的表現取決於具體的反應和應用場景，因此很難單純地說哪一組元素的表現最好。然而，一些研究表明，包含貴金屬如鉑（Pt）和鈀（Pd）的高熵合金在某些催化反應中表現出色，特別是在需要高催化活性和選擇性的反應中。

另一方面，完全由過渡金屬組成的高熵合金（如Ni、Co、Fe、Cu、Cr、Mn等）在成本和穩定性方面具有優勢，並且在某些工業應用中也能提供良好的催化性能。

因此，選擇哪一組元素組合取決於具體的應用需求、成本考量以及所需的催化性能。研究人員通常會根據這些因素進行優化和調整，以達到最佳的催化效果。
